In [1]:
import sys
import gc
from collections import Counter
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
filename = "Nabokov_Vladimir__Lolita_Scenarii__Readli.Net_120895.txt"
_raw_text = open(filename, encoding='utf-8').read()
_raw_text = _raw_text.lower()

In [3]:
len(_raw_text), len(_raw_text.replace('\n\n', ' ').replace('\n\n\n', ' '))

(329634, 326012)

In [4]:
# 50000, -30000
# print(raw_text[-30000:-10000])

In [5]:
raw_text = _raw_text[50000:-30000].replace('\n\n', ' ')

In [6]:
lettersc = Counter(raw_text)

In [7]:
lettersc.most_common(20)

[(' ', 37057),
 ('о', 21015),
 ('е', 17152),
 ('а', 16064),
 ('т', 14918),
 ('и', 12207),
 ('н', 11920),
 ('р', 9418),
 ('с', 9382),
 ('л', 9049),
 ('в', 7707),
 ('м', 7246),
 ('к', 6496),
 ('у', 6212),
 ('д', 5795),
 ('п', 5051),
 ('я', 4382),
 ('г', 4079),
 ('.', 3881),
 ('б', 3840)]

In [8]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [9]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '«', '»', 'è', 'é', 'ó', 'ù', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '—', '„', '…', '№']


In [10]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  247007
Total Vocab:  94


In [11]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  246907


In [12]:
np.array(dataX).shape, np.array(dataY).shape

((246907, 100), (246907,))

In [13]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
# X = X / float(n_vocab)
X = np_utils.to_categorical(X)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [34]:
# for i in range(20):
#     keras.backend.clear_session()
#     # del model
#     gc.collect()

In [14]:
X.shape

(246907, 100, 94)

In [16]:
y.shape

(246907, 94)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100, 256)          359424    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 94)                24158     
Total params: 908,894
Trainable params: 908,894
Non-trainable params: 0
_________________________________________________________________


In [21]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=100, batch_size=512, callbacks=callbacks_list)

Epoch 1/100
130048/246907 [==============>...............] - ETA: 1:23 - loss: 3.2057

# Generating text

In [24]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
def seq_to_text(seq):
    

In [ ]:
np_utils.to_categorical(num_classes=)

In [42]:
# pick a random seed
n = 200
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print('''Seed: "''')

print(''.join([int_to_char[value] for value in pattern]))
print('''"\nGenerated: "''')
# generate characters
for i in range(n):
    x = np.reshape(pattern, (1, len(pattern), 1))
#     x = x / float(n_vocab)
    prediction = model.predict(np_utils.to_categorical(x, num_classes=94), verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('''\nOriginal: "''')
print(raw_text[start:start + n])
print('''"''')

Seed: "
 дверь стенного шкафа, такая же дверь в ванную, чернильно-синее окно, отраженная в нем кровать, та ж
"
Generated: "
е лицо поднимается на подстилке 2, рядом с гумбертом. он принимается на полу, под себя получали их в своем комнаты и лолита на подстилке 2, рядом с гумбертом. лолита: (прочищая горлица подстилка 2) и 
Original: "
 дверь стенного шкафа, такая же дверь в ванную, чернильно-синее окно, отраженная в нем кровать, та же кровать в зеркале шкафа, два кресла, стол со стеклянным верхом, два ночных столика, между ними дву
"


In [33]:
raw_text[146264:146264 + 400]

'опсон, а потом — гумберт. эдда хопсон (ее имя красуется у нее на спине), воспользовавшись тем, что тропа расширилась, поравнялась с гумбертом и вовлекает его — принужденного — в светский разговор (о, тень шарлотты!). миссис хопсон: ах, что за прелестный у вас ребенок! я восхищалась ею вчера вечером в холле. эти скулы! этот девственный пушок на руках и ногах! я немного художник, у меня даже были вы'

In [ ]:
# При представлении букв в виде цифр и 100 эпохах loss был на уровне 2.4247
# А текст на уровне:
# Seed:
# " вленной продолжительности. должен напомнить вам, что вы обещали мне прислать в скором времени ваши с "
#  сертй порене. гомбе гумберт: (пооаая ва н семей но норение) по прораелея с пем но но сооеналт воеое
# Done.
